### LSTM downstream prediction model

Evaluate an LSTM downstream prediction model on the raw physiological signal embeddings and the static patient data.

In [1]:
from keras.utils import multi_gpu_model
from keras.layers import Input, LSTM, Dense, Dropout
from keras.models import Sequential, load_model, Model
from keras.optimizers import RMSprop, SGD, Adam
from matplotlib import cm, pyplot as plt
from sklearn import metrics
from os.path import expanduser as eu
from os.path import isfile, join
from os import listdir
import sklearn.metrics as metrics
import numpy as np
import random
import time
import keras

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "6"
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto(allow_soft_placement=True,gpu_options = tf.GPUOptions(allow_growth=True))
set_session(tf.Session(config=config))
GPUNUM = len(os.environ["CUDA_VISIBLE_DEVICES"].split(","))

def standardize_static(X_lst):
    mean = X_lst[-1].mean(0)
    std  = X_lst[-1].std(0)
    std[std == 0] = 1
    X_lst[-1] = (X_lst[-1] - mean)/std
    return(X_lst)

def load_min_model_helper(MPATH):
    print("[PROGRESS] Starting load_min_model_helper()")
    print("[DEBUG] MPATH {}".format(MPATH))
    mfiles = os.listdir(MPATH)
    full_mod_name = MPATH.split("/")[-1]
    mfiles = [f for f in mfiles if "val_loss" in f]
    loss_lst = [float(f.split("val_loss:")[1].split("_")[0]) for f in mfiles]
    min_ind = loss_lst.index(min(loss_lst))
    min_mod_name = "{}/{}".format(MPATH,mfiles[min_ind])
    if DEBUG: print("[DEBUG] min_mod_name {}".format(mfiles[min_ind]))
    return(load_model(min_mod_name))

def split_data_lst(X_trval_lst,y_trval):
    train_ratio = 0.9
    sample_size = X_trval_lst[0].shape[0]
    nine_tenths_ind = int(train_ratio*sample_size)
    X_train = [X[0:nine_tenths_ind,:] for X in X_trval_lst]
    y_train = y_trval[0:nine_tenths_ind]
    X_valid = [X[nine_tenths_ind:sample_size,:] for X in X_trval_lst]
    y_valid = y_trval[nine_tenths_ind:sample_size]
    del X_trval_lst
    gc.collect()
    return(X_train, y_train, X_valid, y_valid)

def form_train_model(X_trval_lst,y_trval,hyper_dict,
                     label_type,epoch_num=50,is_tune=True):
    ########## Form Data #########
    X_train_lst, y_train, X_valid_lst, y_valid = split_data_lst(X_trval_lst,y_trval)

    X_train_lst = [X[:,:,np.newaxis] if X.shape[1] == 60 else X for X in X_train_lst]
    X_train_lst = [np.concatenate(X_train_lst[:-1],2),X_train_lst[-1]]

    X_valid_lst = [X[:,:,np.newaxis] if X.shape[1] == 60 else X for X in X_valid_lst]
    X_valid_lst = [np.concatenate(X_valid_lst[:-1],2),X_valid_lst[-1]]

    ########## Form Model #########
    print("[PROGRESS] form_model()")

    # Hyperparameters
    numlayer = hyper_dict["numlayer"]
    nodesize = hyper_dict["numnode"]
    opt_name = hyper_dict["opt"]
    drop     = hyper_dict["drop"]
    lr       = hyper_dict["lr"]
    
    if opt_name == "RMSprop":
        opt = RMSprop(lr);
    elif opt_name == "Adam":
        opt = Adam(lr);
    elif opt_name == "SGD":
        opt = SGD(lr);
    
    b_size = 1000
    per_epoch_size = 300000

    lookback = 60
    loss_func = "binary_crossentropy"

    # Model name
    if is_tune:
        mod_path  = "tune_multilstm_{}hospdata_{}label".format(hosp_data,label_type)
        mod_name  = "".join(["{}{}_".format(hyper_dict[k],k) for k in hyper_dict])
        mod_name += "{}bsize_{}epochnum_{}perepochsize".format(b_size,epoch_num,per_epoch_size)
        MODDIR = "{}models/{}/{}/".format(PATH,mod_path,mod_name)
    else:
        mod_name  = "multilstm_{}hospdata_{}label_".format(hosp_data,label_type)
        mod_name += "".join(["{}{}_".format(hyper_dict[k],k) for k in hyper_dict])
        mod_name += "{}bsize_{}epochnum_{}perepochsize".format(b_size,epoch_num,per_epoch_size)
        MODDIR = "{}models/{}/".format(PATH,mod_name)
    
    input_lst   = []; encoded_lst = []
    # Signals
    sig = Input(shape=(lookback,15))
    if numlayer == 1:
        lstm1 = LSTM(nodesize, recurrent_dropout=drop)
        encoded = lstm1(sig)
    elif numlayer == 2:
        lstm1 = LSTM(nodesize, recurrent_dropout=drop, return_sequences=True)
        lstm2 = LSTM(nodesize, recurrent_dropout=drop, dropout=drop)
        encoded = lstm2(lstm1(sig))
    elif numlayer == 3:
        lstm1 = LSTM(nodesize, recurrent_dropout=drop, return_sequences=True)
        lstm2 = LSTM(nodesize, recurrent_dropout=drop, dropout=drop, return_sequences=True)
        lstm3 = LSTM(nodesize, recurrent_dropout=drop, dropout=drop)
        encoded = lstm3(lstm2(lstm1(sig)))
    else:
        assert numlayer <= 3, "Too many layers"
    input_lst.append(sig); encoded_lst.append(encoded)

    # Static variables
    static_size = X_trval_lst[-1].shape[1]
    static = Input(shape=[static_size])
    input_lst   += [static]; encoded_lst += [static]
    
    # Combine and compile model
    merged_vector = keras.layers.concatenate(encoded_lst, axis=-1)
    predictions = Dense(1, activation='sigmoid')(merged_vector)
    model = Model(inputs=input_lst, outputs=predictions)
    model.compile(optimizer=opt, loss=loss_func)
    
    ########## Training #########
    print("[PROGRESS] Starting train_model()")

    loss_keys = ["loss", 'val_loss']

    print("[PROGRESS] Making MODDIR {}".format(MODDIR))
    if not os.path.exists(MODDIR): os.makedirs(MODDIR)
    with open(MODDIR+"loss.txt", "w") as f:
        f.write("\t".join(["i", "epoch_time"] + loss_keys)+"\n")

    # Train and Save
    start_time = time.time()
    for i in range(0,epoch_num):
        train_subset_inds = np.random.choice(X_train_lst[0].shape[0],per_epoch_size,replace=False)
#         pos_inds = np.where(y_train==1)[0]
#         neg_inds = np.where(y_train!=1)[0]
#         neg_subset_inds = np.random.choice(neg_inds,pos_inds.shape[0],replace=False)
#         train_subset_inds = np.concatenate([pos_inds,neg_subset_inds])
        np.random.shuffle(train_subset_inds)
        X_train_lst_sub = [X[train_subset_inds] for X in X_train_lst]
        y_train_sub     = y_train[train_subset_inds]
        history = model.fit(X_train_lst_sub, y_train_sub, epochs=1, batch_size=b_size, 
                            validation_data=(X_valid_lst,y_valid))

        # Save details about training
        epoch_time = time.time() - start_time
        write_lst = [i, epoch_time]
        write_lst += [history.history[k][0] for k in loss_keys]
        with open(MODDIR+"loss.txt", "a") as f:
            f.write("\t".join([str(round(e,5)) for e in write_lst])+"\n")

        # Save model each iteration
        val_loss = history.history['val_loss'][0]
        model.save("{}val_loss:{}_epoch:{}.h5".format(MODDIR,val_loss,i))

    return(MODDIR)

def load_model_and_test(RESDIR,MODDIR,X_test,y_test,data_type,hosp_data):
    model = load_min_model_helper(MODDIR)
    save_path = RESDIR+"hosp{}_data/{}/".format(hosp_data,data_type)
    if not os.path.exists(save_path): os.makedirs(save_path)
    print("[DEBUG] Loading model from {}".format(save_path))
    ypred = model.predict(X_test)
    np.save(save_path+"ypred.npy",ypred)
    np.save(save_path+"y_test.npy",y_test)
    auc = metrics.average_precision_score(y_test, ypred)
    np.random.seed(231)
    auc_lst = []
    roc_auc_lst = []
    for i in range(0,100):
        inds = np.random.choice(X_test[-1].shape[0], X_test[-1].shape[0], replace=True)
        auc = metrics.average_precision_score(y_test[inds], ypred[inds])
        auc_lst.append(auc)
        roc_auc = metrics.roc_auc_score(y_test[inds], ypred[inds])
        roc_auc_lst.append(roc_auc)
    auc_lst = np.array(auc_lst)
    roc_auc_lst = np.array(roc_auc_lst)
    print("[DEBUG] auc_lst.mean(): {}".format(auc_lst.mean()))
    print("[DEBUG] roc_auc_lst.mean(): {}".format(roc_auc_lst.mean()))

    SP = RESDIR+"hosp{}_data/".format(hosp_data)
    f = open('{}conf_int_hospdata{}_prauc.txt'.format(SP,hosp_data),'a')
    f.write("{}, {}+-{}\n".format(data_type,auc_lst.mean().round(4),2*np.std(auc_lst).round(4)))
    f.close()
    f = open('{}conf_int_hospdata{}_rocauc.txt'.format(SP,hosp_data),'a')
    f.write("{}, {}+-{}\n".format(data_type,roc_auc_lst.mean().round(4),2*np.std(roc_auc_lst).round(4)))
    f.close()
    np.save("{}auc_lst".format(save_path,data_type), auc_lst)
    np.save("{}roc_auc_lst".format(save_path,data_type), roc_auc_lst)

Using TensorFlow backend.


## Hyperparameter tuning

In [ ]:
import sys
sys.path.append("..")
from xgb_setup import *
os.nice(5)
PATH = "/homes/gws/hughchen/phase/downstream_prediction/"
DPATH = PATH
RESULTPATH = PATH+"/results/"
MODELPATH  = PATH+"/models/"
DEBUG = False

# label_type_eta_currfeat_lst = [("desat_bool92_5_nodesat",0.02,"SAO2"),
#                                ("nibpm60",0.1,"NIBPM"), 
#                                ("etco235",0.1,"ETCO2")]

label_type = "desat_bool92_5_nodesat"
curr_feat = "SAO2"
print("\n[Progress] label_type: {}, curr_feat {}".format(label_type, curr_feat))

hosp_data = 0
data_type = "proc[top15]+nonsignal"

# Load train validation data and split it
(X_trval_lst,y_trval) = load_data(DPATH,data_type,label_type,True,hosp_data,curr_feat,DEBUG=DEBUG)
X_trval_lst = standardize_static(X_trval_lst)

numlayers = [1,2,3]
numnodes  = [100,200,300]
opt_lst   = ["RMSprop", "SGD", "Adam"]
lr_lst    = [0.01,0.001,0.0001]
drop_lst  = [0,0.5]

for numlayer in numlayers:
    for numnode in numnodes:
        for opt in opt_lst[0:1]:
            for lr in lr_lst:
                for drop in drop_lst:
                    hyper_dict = {"numlayer":numlayer,"numnode":numnode,
                                  "opt":opt,"lr":lr,"drop":drop}
                    form_train_model(X_trval_lst,y_trval,hyper_dict,label_type)

## Load min model and train and evaluate

In [ ]:
import sys
sys.path.append("..")
from xgb_setup import *
os.nice(5)
PATH = "/homes/gws/hughchen/phase/downstream_prediction/"
DPATH = PATH
RESULTPATH = PATH+"/results/"
MODELPATH  = PATH+"/models/"
DEBUG = False

label_type = "desat_bool92_5_nodesat"
curr_feat = "SAO2"
print("\n[Progress] label_type: {}, curr_feat {}".format(label_type, curr_feat))

hosp_data = 0
data_type = "proc[top15]+nonsignal"

# Get best model in terms of min validation loss
TUNEPATH = MODELPATH+"tune_multilstm_0hospdata_desat_bool92_5_nodesatlabel/"
mod_names = os.listdir(TUNEPATH)
best_min_val_loss = float("inf")
for mod_name in mod_names:
    CURRMODPATH = TUNEPATH+mod_name
    model_checkpts = [f for f in os.listdir(CURRMODPATH) if "val_loss" in f]
    min_val_loss = float(sorted(model_checkpts)[1].split("val_loss:")[1].split("_")[0])
    if min_val_loss < best_min_val_loss:
        best_min_val_loss = min_val_loss
        best_mod_name = mod_name

hyper_dict = {"numlayer" : int(best_mod_name.split("numlayer_")[0]),
              "numnode"  : int(best_mod_name.split("numnode_")[0].split("_")[-1]),
              "opt"      : best_mod_name.split("opt_")[0].split("_")[-1],
              "lr"       : float(best_mod_name.split("lr_")[0].split("_")[-1]),
              "drop"     : float(best_mod_name.split("drop_")[0].split("_")[-1])}

# Load train validation data and split it
(X_trval_lst,y_trval) = load_data(DPATH,data_type,label_type,True,hosp_data,curr_feat,DEBUG=DEBUG)
X_trval_lst = standardize_static(X_trval_lst)

# Train model
MODDIR = form_train_model(X_trval_lst,y_trval,hyper_dict,label_type,is_tune=False,epoch_num=200)

mod_type = "multilstm_{}".format(label_type)
RESDIR = '{}{}/'.format(RESULTPATH, mod_type)
if not os.path.exists(RESDIR): os.makedirs(RESDIR)

# Load test data
(X_test1_lst,y_test1) = load_data(DPATH,data_type,label_type,False,hosp_data,curr_feat,DEBUG=DEBUG)
X_test1_lst = standardize_static(X_test1_lst)
X_test1_lst = [X[:,:,np.newaxis] if X.shape[1] == 60 else X for X in X_test1_lst]
X_test1_lst = [np.concatenate(X_test1_lst[:-1],2),X_test1_lst[-1]]

# Evaluate on the test set
load_model_and_test(RESDIR,MODDIR,X_test1_lst,y_test,data_type,hosp_data)


[Progress] label_type: desat_bool92_5_nodesat, curr_feat SAO2
[DEBUG] Y.shape: (3920564,)
[DEBUG] Starting load_raw_data
[DEBUG] DPATH /homes/gws/hughchen/phase/downstream_prediction//data/desat_bool92_5_nodesat/hospital_0/


W1028 09:03:05.937915 140596101756736 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1028 09:03:05.939880 140596101756736 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1028 09:03:05.943795 140596101756736 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.



[PROGRESS] form_model()


W1028 09:03:19.665680 140596101756736 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W1028 09:03:19.672633 140596101756736 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3376: The name tf.log is deprecated. Please use tf.math.log instead.

W1028 09:03:19.677962 140596101756736 deprecation.py:323] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[PROGRESS] Starting train_model()
[PROGRESS] Making MODDIR /homes/gws/hughchen/phase/downstream_prediction/models/multilstm_0hospdata_desat_bool92_5_nodesatlabel_3numlayer_300numnode_RMSpropopt_0.001lr_0.0drop_1000bsize_200epochnum_300000perepochsize/


W1028 09:03:23.311719 140596101756736 deprecation_wrapper.py:119] From /homes/gws/hughchen/anaconda2/envs/tf36/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:986: The name tf.assign_add is deprecated. Please use tf.compat.v1.assign_add instead.



Train on 300000 samples, validate on 392057 samples
Epoch 1/1
300000/300000 [==============================] - 171s 570us/step - loss: 0.0550 - val_loss: 0.0437
Train on 300000 samples, validate on 392057 samples
Epoch 1/1
300000/300000 [==============================] - 166s 554us/step - loss: 0.0449 - val_loss: 0.0410
Train on 300000 samples, validate on 392057 samples
Epoch 1/1
300000/300000 [==============================] - 166s 552us/step - loss: 0.0407 - val_loss: 0.0396
Train on 300000 samples, validate on 392057 samples
Epoch 1/1
299000/300000 [============================>.] - ETA: 0s - loss: 0.0412

In [6]:
epoch_num=200
is_tune=False
b_size = 1000
per_epoch_size = 300000
lookback = 60
loss_func = "binary_crossentropy"

# Model name
if is_tune:
    mod_path  = "tune_multilstm_{}hospdata_{}label".format(hosp_data,label_type)
    mod_name  = "".join(["{}{}_".format(hyper_dict[k],k) for k in hyper_dict])
    mod_name += "{}bsize_{}epochnum_{}perepochsize".format(b_size,epoch_num,per_epoch_size)
    MODDIR = "{}models/{}/{}/".format(PATH,mod_path,mod_name)
else:
    mod_name  = "multilstm_{}hospdata_{}label_".format(hosp_data,label_type)
    mod_name += "".join(["{}{}_".format(hyper_dict[k],k) for k in hyper_dict])
    mod_name += "{}bsize_{}epochnum_{}perepochsize".format(b_size,epoch_num,per_epoch_size)
    MODDIR = "{}models/{}/".format(PATH,mod_name)
    
mod_type = "multilstm_{}".format(label_type)
RESDIR = '{}{}/'.format(RESULTPATH, mod_type)
if not os.path.exists(RESDIR): os.makedirs(RESDIR)
    
# Load test data
(X_test1_lst,y_test1) = load_data(DPATH,data_type,label_type,False,hosp_data,curr_feat,DEBUG=DEBUG)
X_test1_lst = standardize_static(X_test1_lst)
X_test1_lst = [X[:,:,np.newaxis] if X.shape[1] == 60 else X for X in X_test1_lst]
X_test1_lst = [np.concatenate(X_test1_lst[:-1],2),X_test1_lst[-1]]

# Evaluate on the test set
load_model_and_test(RESDIR,MODDIR,X_test1_lst,y_test1,data_type,hosp_data)

[DEBUG] Y.shape: (493398,)
[DEBUG] Starting load_raw_data
[DEBUG] DPATH /homes/gws/hughchen/phase/downstream_prediction//data/desat_bool92_5_nodesat/hospital_0/
[PROGRESS] Starting load_min_model_helper()
[DEBUG] MPATH /homes/gws/hughchen/phase/downstream_prediction/models/multilstm_0hospdata_desat_bool92_5_nodesatlabel_3numlayer_300numnode_RMSpropopt_0.001lr_0.0drop_1000bsize_200epochnum_300000perepochsize/
[DEBUG] Loading model from /homes/gws/hughchen/phase/downstream_prediction//results/multilstm_desat_bool92_5_nodesat/hosp0_data/proc[top15]+nonsignal/
[DEBUG] auc_lst.mean(): 0.27563223488754607
[DEBUG] roc_auc_lst.mean(): 0.911993681631015


In [ ]:
import sys
sys.path.append("..")
from xgb_setup import *
os.nice(5)
PATH = "/homes/gws/hughchen/phase/downstream_prediction/"
DPATH = PATH
RESULTPATH = PATH+"/results/"
MODELPATH  = PATH+"/models/"
DEBUG = False

label_type = "desat_bool92_5_nodesat"
curr_feat = "SAO2"
print("\n[Progress] label_type: {}, curr_feat {}".format(label_type, curr_feat))

hosp_data = 1
data_type = "proc[top15]+nonsignal"

# Get best model in terms of min validation loss
TUNEPATH = MODELPATH+"tune_multilstm_0hospdata_desat_bool92_5_nodesatlabel/"
mod_names = os.listdir(TUNEPATH)
best_min_val_loss = float("inf")
for mod_name in mod_names:
    CURRMODPATH = TUNEPATH+mod_name
    model_checkpts = [f for f in os.listdir(CURRMODPATH) if "val_loss" in f]
    min_val_loss = float(sorted(model_checkpts)[1].split("val_loss:")[1].split("_")[0])
    if min_val_loss < best_min_val_loss:
        best_min_val_loss = min_val_loss
        best_mod_name = mod_name

hyper_dict = {"numlayer" : int(best_mod_name.split("numlayer_")[0]),
              "numnode"  : int(best_mod_name.split("numnode_")[0].split("_")[-1]),
              "opt"      : best_mod_name.split("opt_")[0].split("_")[-1],
              "lr"       : float(best_mod_name.split("lr_")[0].split("_")[-1]),
              "drop"     : float(best_mod_name.split("drop_")[0].split("_")[-1])}

# Load train validation data and split it
(X_trval_lst,y_trval) = load_data(DPATH,data_type,label_type,True,hosp_data,curr_feat,DEBUG=DEBUG)
X_trval_lst = standardize_static(X_trval_lst)

# Train model
MODDIR = form_train_model(X_trval_lst,y_trval,hyper_dict,label_type,is_tune=False,epoch_num=200)

mod_type = "multilstm_{}".format(label_type)
RESDIR = '{}{}/'.format(RESULTPATH, mod_type)
if not os.path.exists(RESDIR): os.makedirs(RESDIR)

# Load test data
(X_test1_lst,y_test1) = load_data(DPATH,data_type,label_type,False,hosp_data,curr_feat,DEBUG=DEBUG)
X_test1_lst = standardize_static(X_test1_lst)
X_test1_lst = [X[:,:,np.newaxis] if X.shape[1] == 60 else X for X in X_test1_lst]
X_test1_lst = [np.concatenate(X_test1_lst[:-1],2),X_test1_lst[-1]]

# Evaluate on the test set
load_model_and_test(RESDIR,MODDIR,X_test1_lst,y_test1,data_type,hosp_data)


[Progress] label_type: desat_bool92_5_nodesat, curr_feat SAO2
[DEBUG] Y.shape: (4167959,)
[DEBUG] Starting load_raw_data
[DEBUG] DPATH /homes/gws/hughchen/phase/downstream_prediction//data/desat_bool92_5_nodesat/hospital_1/
[PROGRESS] form_model()
[PROGRESS] Starting train_model()
[PROGRESS] Making MODDIR /homes/gws/hughchen/phase/downstream_prediction/models/multilstm_1hospdata_desat_bool92_5_nodesatlabel_3numlayer_300numnode_RMSpropopt_0.001lr_0.0drop_1000bsize_200epochnum_300000perepochsize/
Train on 300000 samples, validate on 416796 samples
Epoch 1/1
300000/300000 [==============================] - 175s 583us/step - loss: 0.0987 - val_loss: 0.0934
Train on 300000 samples, validate on 416796 samples
Epoch 1/1
300000/300000 [==============================] - 171s 569us/step - loss: 0.0870 - val_loss: 0.0864
Train on 300000 samples, validate on 416796 samples
Epoch 1/1
300000/300000 [==============================] - 173s 576us/step - loss: 0.0838 - val_loss: 0.0861
Train on 300000 

In [9]:
load_model_and_test(RESDIR,MODDIR,X_test1_lst,y_test1,data_type,hosp_data)

[PROGRESS] Starting load_min_model_helper()
[DEBUG] MPATH /homes/gws/hughchen/phase/downstream_prediction/models/multilstm_1hospdata_desat_bool92_5_nodesatlabel_3numlayer_300numnode_RMSpropopt_0.001lr_0.0drop_1000bsize_200epochnum_300000perepochsize/
[DEBUG] Loading model from /homes/gws/hughchen/phase/downstream_prediction//results/multilstm_desat_bool92_5_nodesat/hosp1_data/proc[top15]+nonsignal/
[DEBUG] auc_lst.mean(): 0.21062793170203722
[DEBUG] roc_auc_lst.mean(): 0.8531248926355575
